# Instalamos e importamos librerías

In [80]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u252-b09-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Creamos el Spark Context

In [0]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Organización de Datos 75.06 . Primer Cuatrimestre de 2019. Examen parcial, segunda oportunidad:         
# Ejercicio SPARK


# 1) ACARA posee información histórica sobre la venta de autos 0km           en la Argentina. Posee un RDD con información de cada modelo           (marca, modelo, motor, transmisión, origen)y otro con        la información de ventas (marca, modelo, fecha,       concesionario) . 
# Se desea conocer, para los modelos de origen nacional, cuales son           los modelos que ya se discontinuaron (un modelo discontinuado es          aquel que no tuvo ventas en los últimos 12 meses), obteniendo un            nuevo RDD con (marca, modelo, total_vendido,      fecha_inicio_venta, fecha_discontinuación) , donde la    fecha de discontinuación es la fecha cuando se vendió el último auto            de ese modelo, ordenado ascendentemente por esta fecha. (***) (          15pts) 
 
Aclaración: Se puede asumir que el primer RDD tiene un único           registro para cada Marca y Modelo. 

In [0]:
# primero, definimos dos RDDs modelos para trabajar
ventas = [
    ('ford', 'fiesta', '2015/10/10', 'leman'),
    ('ford', 'fiesta', '2019/10/10', 'suarez'),
    ('ford', 'fiesta', '2015/10/10', 'suarez'),
    ('ford', 'fiesta', '2016/10/10', 'suarez'),
    ('ford', 'ecoSport', '2010/10/10','leman'),
     ('ford', 'ecoSport', '2010/10/10','leman'),
      ('ford', 'ecoSport', '2016/09/10','leman'),
    ('chevrolet', 'cruze', '2019/10/10','leman'),
    ('chevrolet', 'cruze', '2019/10/10', 'suarez'),
    ('chevrolet', 'cruze', '2019/11/10', 'suarez'),
    ('chevrolet', 'astra', '2019/06/12','leman'),
    ('chevrolet', 'cruze', '2020/01/01', 'suarez'),
    ('chevrolet', 'astra', '2015/06/14', 'leman'),
    ('chevrolet', 'astra', '2017/06/14', 'leman'),
    ('chevrolet', 'astra', '2016/06/14', 'suarez'),
    ('volkswagen', 'gol', '2001/11/10', 'suarez'),
    ('volkswagen', 'gol', '2002/11/10', 'suarez'),
    ('volkswagen', 'gol', '2003/11/10', 'suarez'),
    ('volkswagen', 'vento', '2020/02/01','leman'),
    ('volkswagen', 'vento', '2020/01/01','suarez'),
]

modelo = [
    ('ford', 'fiesta', 'regular','manual', 'argentina'),
    ('ford', 'ecoSport', 'regular','automatica', 'brasil'),
    ('ford', 'falcon', 'regular','manual', 'chile'),
    ('chevrolet', 'cruze', 'mejorado','automatica', 'usa'),
    ('chevrolet', 'astra', 'mejorado','automatica', 'argentina'),
    ('volkswagen', 'gol', 'mejorado','manual', 'argentina'),
    ('volkswagen', 'vento', 'super','automatico', 'argentina'),
    ('volkswagen', 'subaru', 'mejorado','manual', 'argentina'),
]

In [0]:
ventasRDD = sc.parallelize(ventas)
modeloRDD = sc.parallelize(modelo)

In [122]:
ventasRDD.collect()

[('ford', 'fiesta', '2015/10/10', 'leman'),
 ('ford', 'fiesta', '2019/10/10', 'suarez'),
 ('ford', 'fiesta', '2015/10/10', 'suarez'),
 ('ford', 'fiesta', '2016/10/10', 'suarez'),
 ('ford', 'ecoSport', '2010/10/10', 'leman'),
 ('ford', 'ecoSport', '2010/10/10', 'leman'),
 ('ford', 'ecoSport', '2016/09/10', 'leman'),
 ('chevrolet', 'cruze', '2019/10/10', 'leman'),
 ('chevrolet', 'cruze', '2019/10/10', 'suarez'),
 ('chevrolet', 'cruze', '2019/11/10', 'suarez'),
 ('chevrolet', 'astra', '2019/06/12', 'leman'),
 ('chevrolet', 'cruze', '2020/01/01', 'suarez'),
 ('chevrolet', 'astra', '2015/06/14', 'leman'),
 ('chevrolet', 'astra', '2017/06/14', 'leman'),
 ('chevrolet', 'astra', '2016/06/14', 'suarez'),
 ('volkswagen', 'gol', '2001/11/10', 'suarez'),
 ('volkswagen', 'gol', '2002/11/10', 'suarez'),
 ('volkswagen', 'gol', '2003/11/10', 'suarez'),
 ('volkswagen', 'vento', '2020/02/01', 'leman'),
 ('volkswagen', 'vento', '2020/01/01', 'suarez')]

In [157]:
ventasTotales = ventasRDD.map(lambda x: ((x[0],x[1]) , (x[2])))
ventasTotales.collect()

[(('ford', 'fiesta'), '2015/10/10'),
 (('ford', 'fiesta'), '2019/10/10'),
 (('ford', 'fiesta'), '2015/10/10'),
 (('ford', 'fiesta'), '2016/10/10'),
 (('ford', 'ecoSport'), '2010/10/10'),
 (('ford', 'ecoSport'), '2010/10/10'),
 (('ford', 'ecoSport'), '2016/09/10'),
 (('chevrolet', 'cruze'), '2019/10/10'),
 (('chevrolet', 'cruze'), '2019/10/10'),
 (('chevrolet', 'cruze'), '2019/11/10'),
 (('chevrolet', 'astra'), '2019/06/12'),
 (('chevrolet', 'cruze'), '2020/01/01'),
 (('chevrolet', 'astra'), '2015/06/14'),
 (('chevrolet', 'astra'), '2017/06/14'),
 (('chevrolet', 'astra'), '2016/06/14'),
 (('volkswagen', 'gol'), '2001/11/10'),
 (('volkswagen', 'gol'), '2002/11/10'),
 (('volkswagen', 'gol'), '2003/11/10'),
 (('volkswagen', 'vento'), '2020/02/01'),
 (('volkswagen', 'vento'), '2020/01/01')]

In [158]:
modeloRDD.collect()

[('ford', 'fiesta', 'regular', 'manual', 'argentina'),
 ('ford', 'ecoSport', 'regular', 'automatica', 'brasil'),
 ('ford', 'falcon', 'regular', 'manual', 'chile'),
 ('chevrolet', 'cruze', 'mejorado', 'automatica', 'usa'),
 ('chevrolet', 'astra', 'mejorado', 'automatica', 'argentina'),
 ('volkswagen', 'gol', 'mejorado', 'manual', 'argentina'),
 ('volkswagen', 'vento', 'super', 'automatico', 'argentina'),
 ('volkswagen', 'subaru', 'mejorado', 'manual', 'argentina')]

In [159]:
# Nos quedamos con los modelos nacionales
nacionales = modeloRDD.map(lambda x: ((x[0],x[1]) , (x[4]))).filter(lambda x: x[1]=='argentina').cache()
nacionales.collect()

[(('ford', 'fiesta'), 'argentina'),
 (('chevrolet', 'astra'), 'argentina'),
 (('volkswagen', 'gol'), 'argentina'),
 (('volkswagen', 'vento'), 'argentina'),
 (('volkswagen', 'subaru'), 'argentina')]

In [160]:
# Eliminamos el campo que almacena el pais de origen
nacionales = nacionales.map(lambda x: ((x[0][0],x[0][1]),1))
nacionales.collect()

[(('ford', 'fiesta'), 1),
 (('chevrolet', 'astra'), 1),
 (('volkswagen', 'gol'), 1),
 (('volkswagen', 'vento'), 1),
 (('volkswagen', 'subaru'), 1)]

In [161]:
# Filtramos las ventas ocurridas hace mas de un ano desd la fecha '2019/06/13'
discontinuados = ventasTotales.filter(lambda x: (x[1][0])<('2019/06/13'))
discontinuados.collect()

[(('ford', 'fiesta'), '2015/10/10'),
 (('ford', 'fiesta'), '2019/10/10'),
 (('ford', 'fiesta'), '2015/10/10'),
 (('ford', 'fiesta'), '2016/10/10'),
 (('ford', 'ecoSport'), '2010/10/10'),
 (('ford', 'ecoSport'), '2010/10/10'),
 (('ford', 'ecoSport'), '2016/09/10'),
 (('chevrolet', 'cruze'), '2019/10/10'),
 (('chevrolet', 'cruze'), '2019/10/10'),
 (('chevrolet', 'cruze'), '2019/11/10'),
 (('chevrolet', 'astra'), '2019/06/12'),
 (('chevrolet', 'cruze'), '2020/01/01'),
 (('chevrolet', 'astra'), '2015/06/14'),
 (('chevrolet', 'astra'), '2017/06/14'),
 (('chevrolet', 'astra'), '2016/06/14'),
 (('volkswagen', 'gol'), '2001/11/10'),
 (('volkswagen', 'gol'), '2002/11/10'),
 (('volkswagen', 'gol'), '2003/11/10'),
 (('volkswagen', 'vento'), '2020/02/01'),
 (('volkswagen', 'vento'), '2020/01/01')]

In [166]:
# Unificamos los RDDs 
discontinuadosNacionales  = nacionales.join(discontinuados)
discontinuadosNacionales.collect()
# Aclaracion: la funcion lambda devuelve 3 campos: el primero, es la suma del anterior + el siguiente;, el 2do campo corresponde
# a la fecha menor y el 3er campo a la fecha mayor.
discontinuadosNacionalesConFecha = discontinuadosNacionales.reduceByKey(lambda x,y: (x[0]+y[0], x[1], y[1]) if x[1]<y[1] else (x[0]+y[0], y[1], x[1]))
discontinuadosNacionalesConFecha.collect()

[(('volkswagen', 'gol'), (3, '2001/11/10', '2003/11/10')),
 (('ford', 'fiesta'), (4, '2015/10/10', '2016/10/10')),
 (('volkswagen', 'vento'), (2, '2020/01/01', '2020/02/01')),
 (('chevrolet', 'astra'), (4, '2015/06/14', '2016/06/14'))]

In [169]:
# mapeamos las tuplas en un RDD de registros
discontinuadosReg = discontinuadosNacionalesConFecha.map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1], x[1][2]))
discontinuadosReg.collect()


[('volkswagen', 'gol', 3, '2001/11/10', '2003/11/10'),
 ('ford', 'fiesta', 4, '2015/10/10', '2016/10/10'),
 ('volkswagen', 'vento', 2, '2020/01/01', '2020/02/01'),
 ('chevrolet', 'astra', 4, '2015/06/14', '2016/06/14')]

In [178]:
# ordenamos de menor a mayor por fecha numero de ventas.. deberia ser por fecha de discontinuacion pero no se como hacerlo con string
discontinuadosReg.sortBy(lambda x:-x[2]).collect()

[('ford', 'fiesta', 4, '2015/10/10', '2016/10/10'),
 ('chevrolet', 'astra', 4, '2015/06/14', '2016/06/14'),
 ('volkswagen', 'gol', 3, '2001/11/10', '2003/11/10'),
 ('volkswagen', 'vento', 2, '2020/01/01', '2020/02/01')]